In [19]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline
from luketools.imports import *

-== luketools/imports.py ==-


# Ivy Deep Learning Framework
## 💎 Vision

Colab notebook 💎 [Vision.ipynb](https://colab.research.google.com/drive/1i5ZU9Uw_OW0HiyCAikMPILMT-7MQ3bDA)

In [33]:
# reqs
!pip install ivy-core
!pip install ivy-vision
!pip install tensorflow --user
!pip install tensorflow-addons --user
!pip install tensorflow-probability --user # added 2022-04-20
!pip install opencv-python
!pip install GoogleDriveDownloader

ERROR: Invalid requirement: '#'


In [10]:
import ivy
import cv2
import requests
import ivy_vision
import numpy as np
import matplotlib.pyplot as plt
from google_drive_downloader import GoogleDriveDownloader as gdd

%matplotlib inline
plt.rcParams['figure.figsize'] = [30, 15]

### Download Data
* Save to local `./content/`

In [15]:
files = {'cam1_inv_ext_mat.npy': '1BoQCjs1NkhlkcQGOpwD-lMtP0qf4hRJT',
         'cam2_inv_ext_mat.npy': '1to-9fDDJPlq7qUb_KzAsVuPpKrV6_VeA',
         'depth1.png': '1SYB27MfazMB_-3dbK1wLnjShufccy_gd',
         'depth2.png': '1BM7qVJlr-FaG8di8LYx8WlamqIP8Aa_2',
         'rgb1.png': '1Z7RrTBT_nuoOWqUSFbBR_-qgROLVdvtx',
         'rgb2.png': '1fWOlJb-808_bBWGVi0Uejl9pWm44lpr5'}

for fname, file_id in files.items():
    gdd.download_file_from_google_drive(
      file_id=file_id,
      dest_path='./content/' + fname) 

In [25]:
!dir content

 Volume in drive C is OS
 Volume Serial Number is 4E17-70DC

 Directory of C:\python\github\ivy_notebooks\notebooks\content

20/04/2022  13:44    <DIR>          .
20/04/2022  13:44    <DIR>          ..
20/04/2022  13:44               224 cam1_inv_ext_mat.npy
20/04/2022  13:44               224 cam2_inv_ext_mat.npy
20/04/2022  13:44           657,224 depth1.png
20/04/2022  13:44           643,012 depth2.png
20/04/2022  13:44           226,052 rgb1.png
20/04/2022  13:44           257,019 rgb2.png
               6 File(s)      1,783,755 bytes
               2 Dir(s)  218,535,673,856 bytes free


In [38]:
ivy.set_framework('tensorflow')

### 👌 Helper functions

In [39]:
b = 12
bo2 = int(b/2)
s = 512


def flow_to_image(flow):
    u = flow[:, :, 0]
    v = flow[:, :, 1]

    idxUnknow = (abs(u) > 1e7) | (abs(v) > 1e7)
    u[idxUnknow] = 0
    v[idxUnknow] = 0

    rad = np.sqrt(u ** 2 + v ** 2)
    maxrad = max(-1, np.max(rad))

    u = u / (maxrad + np.finfo(float).eps + 1e-12)
    v = v / (maxrad + np.finfo(float).eps + 1e-12)

    [h, w] = u.shape
    img = np.zeros([h, w, 3])
    nanIdx = np.isnan(u) | np.isnan(v)
    u[nanIdx] = 0
    v[nanIdx] = 0

    RY = 15
    YG = 6
    GC = 4
    CB = 11
    BM = 13
    MR = 6

    ncols = RY + YG + GC + CB + BM + MR

    colorwheel = np.zeros([ncols, 3])

    col = 0

    # RY
    colorwheel[0:RY, 0] = 255
    colorwheel[0:RY, 1] = np.transpose(np.floor(255 * np.arange(0, RY) / (RY + 1e-12)))
    col += RY

    # YG
    colorwheel[col:col + YG, 0] = 255 - np.transpose(np.floor(255 * np.arange(0, YG) / (YG + 1e-12)))
    colorwheel[col:col + YG, 1] = 255
    col += YG

    # GC
    colorwheel[col:col + GC, 1] = 255
    colorwheel[col:col + GC, 2] = np.transpose(np.floor(255 * np.arange(0, GC) / (GC + 1e-12)))
    col += GC

    # CB
    colorwheel[col:col + CB, 1] = 255 - np.transpose(np.floor(255 * np.arange(0, CB) / (CB + 1e-12)))
    colorwheel[col:col + CB, 2] = 255
    col += CB

    # BM
    colorwheel[col:col + BM, 2] = 255
    colorwheel[col:col + BM, 0] = np.transpose(np.floor(255 * np.arange(0, BM) / (BM + 1e-12)))
    col += + BM

    # MR
    colorwheel[col:col + MR, 2] = 255 - np.transpose(np.floor(255 * np.arange(0, MR) / (MR + 1e-12)))
    colorwheel[col:col + MR, 0] = 255

    ncols = np.size(colorwheel, 0)
    rad = np.sqrt(u ** 2 + v ** 2)

    a = np.arctan2(-v, -u) / np.pi

    fk = (a + 1) / 2 * (ncols - 1) + 1

    k0 = np.floor(fk).astype(int)

    k1 = k0 + 1
    k1[k1 == ncols + 1] = 1
    f_ = fk - k0

    for i in range(0, np.size(colorwheel, 1)):
        tmp = colorwheel[:, i]
        col0 = tmp[k0 - 1] / 255
        col1 = tmp[k1 - 1] / 255
        col = (1 - f_) * col0 + f_ * col1

        idx = rad <= 1
        col[idx] = 1 - rad[idx] * (1 - col[idx])
        notidx = np.logical_not(idx)

        col[notidx] *= 0.75
        img[:, :, i] = np.uint8(np.floor(255 * col * (1 - nanIdx)))

    idx = np.repeat(idxUnknow[:, :, np.newaxis], 3, axis=2)
    img[idx] = 0

    return img/255


def add_top_left_corner_text(img, text_str, x_len, y_len, t=3, fs=1.0):
    img[0:y_len, 0:x_len] = np.zeros_like(img[0:y_len, 0:x_len])
    img[t:y_len-t, t:x_len-t] = np.ones_like(img[t:y_len-t, t:x_len-t])
    cv2.putText(img, text_str, (15, 30), cv2.FONT_HERSHEY_SIMPLEX, fs, tuple([0] * 3), 2)


def show_rgb_and_depth_images(col1, col2, dep1, dep2, dep_lims):

    final_img = np.zeros((s+2*b, 4*s+2*b, 3))

    col1_img = np.copy(np.flip(ivy.to_numpy(col1), -1))
    add_top_left_corner_text(col1_img, 'color 1', 140, 44, 6)

    col2_img = np.copy(np.flip(ivy.to_numpy(col2), -1))
    add_top_left_corner_text(col2_img, 'color 2', 140, 44, 6)

    dep_min = ivy.to_numpy(dep_lims[0]).reshape(()).item()
    depth_range = ivy.to_numpy(dep_lims[1] - dep_lims[0]).reshape(()).item()

    depth1_normed = (ivy.to_numpy(dep1) - dep_min) / depth_range
    depth1_img = np.tile(depth1_normed, (1, 1, 3))
    add_top_left_corner_text(depth1_img, 'depth 1', 150, 44, 6)

    depth2_normed = (ivy.to_numpy(dep2) - dep_min) / depth_range
    depth2_img = np.tile(depth2_normed, (1, 1, 3))
    add_top_left_corner_text(depth2_img, 'depth 2', 150, 44, 6)

    final_img[b:-b, b:-b] = np.concatenate((col1_img, col2_img, depth1_img, depth2_img), 1)
    final_img[:, s+bo2:s+3*bo2] = 0.
    final_img[:, 2*s+bo2:2*s+3*bo2] = 0.
    final_img[:, 3*s+bo2:3*s+3*bo2] = 0.

    plt.imshow(np.clip(final_img, 0, 1))
    plt.show()


def show_flow_and_depth_images(dep1, f1to2, dep1_from_flow, dep_lims):

    final_img = np.zeros((s+2*b, 3*s+2*b, 3))

    dep_min = ivy.to_numpy(dep_lims[0]).reshape(()).item()
    depth_range = ivy.to_numpy(dep_lims[1] - dep_lims[0]).reshape(()).item()

    depth1_normed = (ivy.to_numpy(dep1) - dep_min) / depth_range
    depth1_img = np.tile(depth1_normed, (1, 1, 3))
    add_top_left_corner_text(depth1_img, 'depth 1', 150, 44, 6)

    flow_img = flow_to_image(np.copy(ivy.to_numpy(f1to2)))
    add_top_left_corner_text(flow_img, 'flow 1 to 2', 215, 44, 6)

    depth1_from_flow_normed = (ivy.to_numpy(dep1_from_flow) - dep_min) / depth_range
    depth1_from_flow_img = np.tile(depth1_from_flow_normed, (1, 1, 3))
    add_top_left_corner_text(depth1_from_flow_img, 'depth 1 from flow', 325, 44, 6)

    final_img[b:-b, b:-b] = np.concatenate((depth1_img, flow_img, depth1_from_flow_img), 1)
    final_img[:, s+bo2:s+3*bo2] = 0.
    final_img[:, 2*s+bo2:2*s+3*bo2] = 0.

    plt.imshow(np.clip(final_img, 0, 1))
    plt.show()


def show_inverse_warped_images(dep1_wrt_f2, dep2_warp_to_f1, dep_val,
                               col1, col2_warp_to_f1, col2_warp_to_f1_masked, dep_lims):

    final_img = np.zeros((2*s+2*b, 3*s+2*b, 3))
    dep_min = ivy.to_numpy(dep_lims[0]).reshape(()).item()
    depth_range = ivy.to_numpy(dep_lims[1] - dep_lims[0]).reshape(()).item()

    dep1_wrt_f2_normed = (ivy.to_numpy(dep1_wrt_f2) - dep_min) / depth_range
    dep1_wrt_f2_img = np.tile(dep1_wrt_f2_normed, (1, 1, 3))
    add_top_left_corner_text(dep1_wrt_f2_img, 'depth 1 wrt f2', 265, 44, 6)

    dep2_warp_to_f1_normed = (ivy.to_numpy(dep2_warp_to_f1) - dep_min) / depth_range
    dep2_warp_to_f1_img = np.tile(dep2_warp_to_f1_normed, (1, 1, 3))
    add_top_left_corner_text(dep2_warp_to_f1_img, 'depth 2 warped to f1', 375, 44, 6)

    dep_val = ivy.to_numpy(dep_val).astype(np.float32)
    dep_val_img = np.tile(dep_val, (1, 1, 3))
    add_top_left_corner_text(dep_val_img, 'depth validity', 240, 44, 6)

    col1_img = np.copy(np.flip(ivy.to_numpy(col1), -1))
    add_top_left_corner_text(col1_img, 'color 1', 140, 44, 6)

    col2_warp_to_f1_img = np.copy(np.flip(ivy.to_numpy(col2_warp_to_f1), -1))
    add_top_left_corner_text(col2_warp_to_f1_img, 'color 2 warped to f1', 360, 44, 6)

    col2_warp_to_f1_masked_img = np.copy(np.flip(ivy.to_numpy(col2_warp_to_f1_masked), -1))
    add_top_left_corner_text(col2_warp_to_f1_masked_img, 'rendering in f1', 175, 44, 6)

    top_half = np.concatenate((dep1_wrt_f2_img, dep2_warp_to_f1_img, dep_val_img), 1)
    bottom_half = np.concatenate((col1_img, col2_warp_to_f1_img, col2_warp_to_f1_masked_img), 1)

    final_img[b:-b, b:-b] = np.concatenate((top_half, bottom_half), 0)
    final_img[:, s+bo2:s+3*bo2] = 0.
    final_img[:, 2*s+bo2:2*s+3*bo2] = 0.
    final_img[:, 3*s+bo2:3*s+3*bo2] = 0.
    final_img[s+bo2:s+3*bo2] = 0.

    plt.imshow(np.clip(final_img, 0, 1))
    plt.show()


def show_forward_warped_images(dep1, col1, f1_f_warp_no_db, f1_f_warp_w_db, dep_lims):

    final_img = np.zeros((2*s+2*b, 3*s+2*b, 3))
    dep_min = ivy.to_numpy(dep_lims[0]).reshape(()).item()
    depth_range = ivy.to_numpy(dep_lims[1] - dep_lims[0]).reshape(()).item()

    dep1_normed = (ivy.to_numpy(dep1) - dep_min) / depth_range
    dep1_img = np.tile(dep1_normed, (1, 1, 3))
    add_top_left_corner_text(dep1_img, 'depth 1', 150, 44, 6)

    dep2_wrt_f1_warped_to_f1_no_db = f1_f_warp_no_db[..., 2:3]
    dep2_wrt_f1_warped_to_f1_no_db_normed = (ivy.to_numpy(dep2_wrt_f1_warped_to_f1_no_db) - dep_min) / depth_range
    dep2_wrt_f1_warped_to_f1_no_db_img = np.tile(dep2_wrt_f1_warped_to_f1_no_db_normed, (1, 1, 3))
    add_top_left_corner_text(dep2_wrt_f1_warped_to_f1_no_db_img, 'depth 2 wr+wrpd to f1 no db', 500, 44, 6)

    dep2_wrt_f1_warped_to_f1_w_db = f1_f_warp_w_db[..., 2:3]
    dep2_wrt_f1_warped_to_f1_w_db_normed = (ivy.to_numpy(dep2_wrt_f1_warped_to_f1_w_db) - dep_min) / depth_range
    dep2_wrt_f1_warped_to_f1_w_db_img = np.tile(dep2_wrt_f1_warped_to_f1_w_db_normed, (1, 1, 3))
    add_top_left_corner_text(dep2_wrt_f1_warped_to_f1_w_db_img, 'depth 2 wr+wrpd to f1 w db', 485, 44, 6)

    col1_img = np.copy(np.flip(ivy.to_numpy(col1), -1))
    add_top_left_corner_text(col1_img, 'color 1', 140, 44, 6)

    col2_warped_to_f1_no_db = f1_f_warp_no_db[..., 3:]
    col2_warp_to_f1_no_db_img = np.copy(np.flip(ivy.to_numpy(col2_warped_to_f1_no_db), -1))
    add_top_left_corner_text(col2_warp_to_f1_no_db_img, 'color 2 warped to f1 no db', 470, 44, 6)

    col2_warped_to_f1_w_db = f1_f_warp_w_db[..., 3:]
    col2_warp_to_f1_w_db_img = np.copy(np.flip(ivy.to_numpy(col2_warped_to_f1_w_db), -1))
    add_top_left_corner_text(col2_warp_to_f1_w_db_img, 'color 2 warped to f1 w db', 455, 44, 6)

    top_half = np.concatenate((dep1_img, dep2_wrt_f1_warped_to_f1_no_db_img, dep2_wrt_f1_warped_to_f1_w_db_img), 1)
    bottom_half = np.concatenate((col1_img, col2_warp_to_f1_no_db_img, col2_warp_to_f1_w_db_img), 1)

    final_img[b:-b, b:-b] = np.concatenate((top_half, bottom_half), 0)
    final_img[:, s+bo2:s+3*bo2] = 0.
    final_img[:, 2*s+bo2:2*s+3*bo2] = 0.
    final_img[:, 3*s+bo2:3*s+3*bo2] = 0.
    final_img[s+bo2:s+3*bo2] = 0.

    plt.imshow(np.clip(final_img, 0, 1))
    plt.show()

[autoreload of ivy failed: Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\site-packages\IPython\extensions\autoreload.py", line 245, in check
    superreload(m, reload, self.old_objects)
  File "C:\ProgramData\Anaconda3\lib\site-packages\IPython\extensions\autoreload.py", line 394, in superreload
    module = reload(module)
  File "C:\ProgramData\Anaconda3\lib\imp.py", line 314, in reload
    return importlib.reload(module)
  File "C:\ProgramData\Anaconda3\lib\importlib\__init__.py", line 148, in reload
    raise ImportError(msg.format(name), name=name)
ImportError: module ivy.tensorflow not in sys.modules
]
